# Engines to test:
### 1. Tesseract
### 2. PaddleOCR
### 3. EasyOCR

## Dataset:

https://guillaumejaume.github.io/FUNSD/

In [ ]:
import os
import json
import pandas as pd
import cv2
import matplotlib.pyplot as plt

annotations_folder = ''
images_folder = ''

data = []

for filename in os.listdir(annotations_folder):
    if filename.endswith('.json'):
        with open(os.path.join(annotations_folder, filename), 'r') as file:
            annotation_data = json.load(file)
            image_filename = filename.split('.')[0] + '.png'
            text = ""
            for item in annotation_data['form']:
                text += item['text'] + " "

            data.append({'filename': image_filename, 'text': text.strip()})

df = pd.DataFrame(data)

df.head()

: 

  Using cached pandas-2.2.2-cp39-cp39-win_amd64.whl.metadata (19 kB)
  Using cached numpy-1.26.4-cp39-cp39-win_amd64.whl.metadata (61 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.2-cp39-cp39-win_amd64.whl (11.6 MB)
Using cached numpy-1.26.4-cp39-cp39-win_amd64.whl (15.8 MB)
Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.1/38.8 MB 825.8 kB/s eta 0:00:47
   ---------------------------------------- 0.3/38.8 MB 2.1 MB/s eta 0:00:19
    --------------------------------------- 0.6/38.8 MB 3.2 MB/s eta 0:00:13
    --------------------------------------- 0.9/38.8 MB 4.2 MB/s eta 0:00:10
   - -------------------------------------- 1.2